In [1]:
import numpy as np
import pandas as pd
import cv2
import tqdm
from matplotlib import pyplot as plt
import pickle

In [2]:
train_i = []
test_i = []

train_xy = pd.read_csv('train.csv')
train_path = train_xy['id'].values
test_path = pd.read_csv('imagenames.csv')['id'].values

for f in train_path:
    i = cv2.imread('./train/' + f + '.jpg')
    train_i.append(i)
for f in test_path:
    i = cv2.imread('./test/' + f + '.jpg')
    test_i.append(i)

In [6]:
sift = cv2.SIFT_create()

# Compute SIFT keypoints and descriptors
for i,img in tqdm.tqdm(enumerate(train_i)):
    _, des = sift.detectAndCompute(img,None)
    f = open(f'./train_kp/train_kp{train_path[i]}.pckl','wb')
    pickle.dump(des,f)
    f.close()

for i,img in tqdm.tqdm(enumerate(test_i)):
    _, des = sift.detectAndCompute(img,None)
    f = open(f'./test_kp/test_kp{test_path[i]}.pckl','wb')
    pickle.dump(des,f)
    f.close()


7500it [11:35, 10.78it/s]
1200it [01:56, 10.31it/s]


In [24]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

best_train = []
second_best_train = []
third_best_train = []
for test in tqdm.tqdm(test_path):

    f = open(f'./test_kp/test_kp{test}.pckl', 'rb')
    des_test = pickle.load(f)
    f.close()

    goods = []
    current_good = 0

    if des_test is None or len(des_test) < 2:
        best_train.append('')
        second_best_train.append('')
        third_best_train.append('')
        continue

    for train in train_path:
        f = open(f'./train_kp/train_kp{train}.pckl', 'rb')
        des_train = pickle.load(f)
        f.close()

        # Matching descriptor using KNN algorithm
        if des_train is None or len(des_train) < 2:
            goods.append(-1)
            continue
        matches = flann.knnMatch(des_test,des_train,k=2)

        # Store all good matches as per Lowe's Ratio test.
        good = len([m for m,n in matches if m.distance < 0.7*n.distance])
        goods.append(good)
        
    best = [train_path[idx] for idx in np.argsort(goods)]
    f = open(f'./best/best_sift_{test}.pckl','wb')
    pickle.dump(best,f)
    f.close()

  0%|          | 1/1200 [04:04<81:33:39, 244.89s/it]

IMG3375_4 IMG3307_5 IMG2832_4
['IMG3734_2', 'IMG3442_3', 'IMG3375_4']


  0%|          | 1/1200 [06:04<121:31:34, 364.88s/it]


KeyboardInterrupt: 